In [1]:
from pyspark.sql import SQLContext

In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

In [3]:
sqlContext=SQLContext(sc)

In [4]:
df=sqlContext.read.load("daily_weather.csv",format="com.databricks.spark.csv",header='true',inferSchema="true")

In [5]:
df.describe().show()

+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|summary|            number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am| rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|  count|              1095|             1092|              1090|                  1091|              1092|                  1092|              1091|                 1089|              1092|                 1095|                 1095|
|   mean|             547.0|918.8825513138094| 64.9330014128

In [10]:
df.count()

1095

In [6]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [7]:
# Remove all the null value rows in the dataset
removeAllDf=df.na.drop()

In [8]:
removeAllDf.count()

1064

In [9]:
removeAllDf.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [11]:
# Get Mean value  after removing missing values

from pyspark.sql.functions import avg

In [53]:
imputeDf=df

for x in imputeDf.columns:
    
    meanvalue=removeAllDf.agg(avg(x)).first()[0]
    print(x,meanvalue)
    imputeDf=imputeDf.na.fill(meanvalue,[x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


In [55]:
df.describe(['air_pressure_9am']).show()
imputeDf.describe(['air_pressure_9am']).show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1095|
|   mean|918.8826078303855|
| stddev|3.179792514510735|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



In [58]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+----------------+------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|number|air_pressure_9am|air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+------+----------------+------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|     0|               3|           5|                     4|                 3|                     3|                 4|                    6|                3|                    0|                    0|
+------+----------------+------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+----------